In [2]:
# 1. 載入必要模組
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm

# 載入自訂模組
%reload_ext autoreload
%autoreload 2
from lib.backtest.strategy_one import backtest_strategy
from lib.performance_analysis import calculate_strategy_one_performance

print("所有模組載入成功。")

# 2. 載入資料
try:
    df_original = pd.read_csv("../data/TPE-sample1.csv", encoding="utf-8")
    print(f"資料載入成功，共 {len(df_original)} 筆。")
except FileNotFoundError:
    print("錯誤：找不到資料檔案 '../data/TPE-sample1.csv'。請確認路徑是否正確。")

# 3. 設定敏感度分析參數
iterations = 100
param_ranges = {
    'ma_period': (3, 15),
    'bb_period': (16, 40),
    'bb_std': (1.0, 3.0),
    'drop_threshold': (0.1, 0.9)
}
results_list = []

print(f"準備進行 {iterations} 次隨機參數測試...")

# 4. 執行敏感度分析迴圈
for _ in tqdm(range(iterations), desc="執行進度"):
    # 生成隨機參數
    ma_p = random.randint(*param_ranges['ma_period'])
    bb_p = random.randint(*param_ranges['bb_period'])
    # 確保 bb_period > ma_period
    while bb_p <= ma_p:
        bb_p = random.randint(*param_ranges['bb_period'])

    bb_s = random.uniform(*param_ranges['bb_std'])
    drop_t = random.uniform(*param_ranges['drop_threshold'])

    # 執行回測
    df_result = backtest_strategy(
        df_original.copy(), # 確保每次都使用原始資料
        ma_period=ma_p,
        bb_period=bb_p,
        bb_std=bb_s,
        drop_threshold=drop_t
    )

    # 計算策略績效
    df_result['entry'] = (df_result['position'] == 1) & (df_result['position'].shift(1) == 0)
    df_result['exit'] = (df_result['position'] == 0) & (df_result['position'].shift(1) == 1)
    performance = calculate_strategy_one_performance(df_result)

    # 儲存結果
    run_results = {
        'ma_period': ma_p,
        'bb_period': bb_p,
        'bb_std': round(bb_s, 2),
        'drop_threshold': round(drop_t, 2)
    }
    run_results.update(performance)
    results_list.append(run_results)

print("敏感度分析完成。")

# 5. 整理與呈現結果
if results_list:
    results_df = pd.DataFrame(results_list)

    # 為了方便閱讀，重新命名績效指標的欄位名稱
    rename_dict = {
        'final_equity': '最終權益',
        'net_profit': '淨利',
        'mdd': '最大回撤(MDD)',
        'total_profit': '總獲利',
        'total_loss': '總虧損',
        'total_trades': '總交易次數',
        'winning_trades': '獲利次數',
        'losing_trades': '虧損次數',
        'win_rate': '勝率',
        'max_profit_per_trade': '單次最大獲利',
        'max_loss_per_trade': '單次最大虧損',
        'avg_profit_per_trade': '平均獲利',
        'avg_loss_per_trade': '平均虧損',
        'profit_loss_ratio': '賺賠比',
        'max_consecutive_wins': '最長連續獲利',
        'max_consecutive_losses': '最長連續虧損'
    }
    results_df.rename(columns=rename_dict, inplace=True)

    # 排序找出最佳結果
    best_results = results_df.sort_values(by='淨利', ascending=False)

    print("\n--- 淨利最高的前10組參數 ---")
    display(best_results.head(10))

    print("\n--- 整體績效統計 ---")
    # 選擇數值型欄位進行描述性統計
    numeric_cols = ['淨利', '最大回撤(MDD)', '總交易次數', '賺賠比']
    display(best_results[numeric_cols].describe())
else:
    print("沒有任何結果可供分析。")

所有模組載入成功。
資料載入成功，共 1261 筆。
準備進行 100 次隨機參數測試...


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html